In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats as stats
import math
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.impute import KNNImputer
from fancyimpute import KNN
from functools import reduce
import plotly.express as px
import plotly.graph_objects as go 
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import plotly.io as pio
pio.renderers.default='notebook'
from sklearn.metrics import (confusion_matrix,
                           accuracy_score)
import warnings # For handling error messages.
# Don't worry about the following two instructions: they just suppress warnings that could occur later. 
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

In [16]:
# We import the dataset that we created in our Data Wrangling section of the Capstone project 
# Import the dataset

df=pd.read_csv('/Users/amrita/Desktop/dummies_data.csv')
df.head()

,satisfaction,Age,Flight_Distance,Seat_comfort,Departure/Arrival_time_convenient,Food_and_drink,Gate_location,Inflight_wifi_service,Inflight_entertainment,Online_support,...,Checkin_service,Cleanliness,Online_boarding,Departure_Delay_in_Minutes,Arrival_Delay_in_Minutes,Eco,Eco_Plus,disloyal_Customer,Personal_Travel,Male
0,1,65,265,0,0,0,2,2,4,2,...,5,3,2,0,0,1,0,0,1,0
1,1,47,2464,0,0,0,3,0,2,2,...,2,3,2,310,305,0,0,0,1,1
2,1,15,2138,0,0,0,3,2,0,2,...,4,4,2,0,0,1,0,0,1,0
3,1,60,623,0,0,0,3,3,4,3,...,4,1,3,0,0,1,0,0,1,0
4,1,70,354,0,0,0,3,4,3,4,...,4,2,5,0,0,1,0,0,1,0


In [17]:
# no missing values
df.isna().sum()

satisfaction                         0
Age                                  0
Flight_Distance                      0
Seat_comfort                         0
Departure/Arrival_time_convenient    0
Food_and_drink                       0
Gate_location                        0
Inflight_wifi_service                0
Inflight_entertainment               0
Online_support                       0
Ease_of_Online_booking               0
On_board_service                     0
Leg_room_service                     0
Baggage_handling                     0
Checkin_service                      0
Cleanliness                          0
Online_boarding                      0
Departure_Delay_in_Minutes           0
Arrival_Delay_in_Minutes             0
Eco                                  0
Eco_Plus                             0
disloyal_Customer                    0
Personal_Travel                      0
Male                                 0
dtype: int64

In [18]:
first_column = df.pop('Departure_Delay_in_Minutes')
df.insert(3, 'Departure_Delay_in_Minutes', first_column)

In [19]:
second_column = df.pop('Arrival_Delay_in_Minutes')
df.insert(4, 'Arrival_Delay_in_Minutes', second_column)

In [20]:
df

,satisfaction,Age,Flight_Distance,Departure_Delay_in_Minutes,Arrival_Delay_in_Minutes,Seat_comfort,Departure/Arrival_time_convenient,Food_and_drink,Gate_location,Inflight_wifi_service,...,Leg_room_service,Baggage_handling,Checkin_service,Cleanliness,Online_boarding,Eco,Eco_Plus,disloyal_Customer,Personal_Travel,Male
0,1,65,265,0,0,0,0,0,2,2,...,0,3,5,3,2,1,0,0,1,0
1,1,47,2464,310,305,0,0,0,3,0,...,4,4,2,3,2,0,0,0,1,1
2,1,15,2138,0,0,0,0,0,3,2,...,3,4,4,4,2,1,0,0,1,0
3,1,60,623,0,0,0,0,0,3,3,...,0,1,4,1,3,1,0,0,1,0
4,1,70,354,0,0,0,0,0,3,4,...,0,2,4,2,5,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129482,1,29,1731,0,0,5,5,5,3,2,...,3,4,4,4,2,1,0,1,1,0
129483,0,63,2087,174,172,2,3,2,4,2,...,3,3,1,2,1,0,0,1,1,1
129484,0,69,2320,155,163,3,0,3,3,3,...,3,4,2,3,2,1,0,1,1,1
129485,0,66,2450,193,205,3,2,3,2,3,...,2,3,2,1,2,1,0,1,1,1


In [23]:
from sklearn.preprocessing import StandardScaler
df.iloc[:, 1:5]= StandardScaler().fit_transform(df.iloc[:, 1:5])

In [24]:
df

,satisfaction,Age,Flight_Distance,Departure_Delay_in_Minutes,Arrival_Delay_in_Minutes,Seat_comfort,Departure/Arrival_time_convenient,Food_and_drink,Gate_location,Inflight_wifi_service,...,Leg_room_service,Baggage_handling,Checkin_service,Cleanliness,Online_boarding,Eco,Eco_Plus,disloyal_Customer,Personal_Travel,Male
0,1,1.691495,-1.671090,-0.386036,-0.392329,0,0,0,2,2,...,0,3,5,3,2,1,0,0,1,0
1,1,0.500825,0.470348,7.786328,7.536854,0,0,0,3,0,...,4,4,2,3,2,0,0,0,1,1
2,1,-1.615922,0.152882,-0.386036,-0.392329,0,0,0,3,2,...,3,4,4,4,2,1,0,0,1,0
3,1,1.360753,-1.322461,-0.386036,-0.392329,0,0,0,3,3,...,0,1,4,1,3,1,0,0,1,0
4,1,2.022237,-1.584420,-0.386036,-0.392329,0,0,0,3,4,...,0,2,4,2,5,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129482,1,-0.689845,-0.243465,-0.386036,-0.392329,5,5,5,3,2,...,3,4,4,4,2,1,0,1,1,0
129483,0,1.559198,0.103217,4.201033,4.079210,2,3,2,4,2,...,3,3,1,2,1,0,0,1,1,1
129484,0,1.956088,0.330117,3.700146,3.845234,3,0,3,3,3,...,3,4,2,3,2,1,0,1,1,1
129485,0,1.757643,0.456714,4.701920,4.937122,3,2,3,2,3,...,2,3,2,1,2,1,0,1,1,1


In [25]:
df.to_csv('new_df.csv',index=False)